# Hybrid System

In [1]:
import pandas as pd
import numpy as np
# from numpy import dot
# from numpy.linalg import norm 

In [2]:
# constants
PATH = '../data/data.csv'

## Import Data

In [3]:
df = pd.read_csv(PATH)
df.shape

(300000, 9)

In [4]:
df.head()

,book_id,author_id,book_genre,reader_id,num_pages,book_rating,publisher_id,publish_year,book_price
0,45,446,2,2272,553,4,10,2004,23
1,2413,23,7,3515,318,2,38,2015,15
2,2095,431,8,18547,208,3,48,2000,23
3,2431,211,4,19675,340,10,34,2000,117
4,2871,78,3,23453,610,2,24,2012,111
